# Connect 4

Proposed solutions:
- **vanilla minimax** with depth of 3 + custom heuristic function.
- **minimax + alpha-beta pruning** using the same configuration as *vanilla minimax*.
- **Monte Carlo Tree Search** using *minimax + alpha-beta pruning* as playout policy (max depth is set to 1 to reduce the execution time).

Across all the proposed solutions, the most expensive task is the computation of the player sequences. In the `Sequences extraction` section different ways to solve this task are analyzed and compared. The most efficient solution exploits (a lot of) caching and the simmetry of all the rows, columns and diagonal on the board.

In [1]:
from typing import List, Callable, Optional, Tuple
from dataclasses import dataclass
from time import perf_counter
import itertools
import functools
import math

from tqdm import tqdm
import numpy as np
np.set_printoptions(precision=2)

from utils import pretty_print_board, counted

In [2]:
NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4

PLAYER_1 = 1
PLAYER_2 = -1

WIN = 1_000_000

# Fix the 'col_height' and 'num_cols' parameters
# of pretty_print_board
pretty_print_board = functools.partial(pretty_print_board,
                                       col_height=COLUMN_HEIGHT,
                                       num_cols=NUM_COLUMNS)


## Basic Functions

Copyright **`(c)`** 2021 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see 'LICENCE.md' for details.

In [3]:
# original unmodified basic functions

def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )

In [4]:
def count_by_length(board: np.ndarray, player: int, length: int) -> int:
    """
    Count the number of sequences of `player` discs having the 
    required `length`

    Parameters
    ----------
    board: np.ndarray
        the connect 4 board
    player: int
        the player whose sequences must be counted
    length: int
        the length of the sequences to count

    Returns
    -------
    int
        the number of sequences of required length
    """
    board = (board == player)

    kernel = np.ones(length)
    diagspan = COLUMN_HEIGHT - length

    sequences = itertools.chain(
        # board rows
        (row for row in board),
        # board columns
        (col for col in board.T),
        # board diagonals
        (np.diag(board, i) for i in range(-diagspan, diagspan + 1)),
        # board antidiagonals
        (np.diag(np.fliplr(board), i) for i in range(-diagspan, diagspan + 1))
    )

    # use convolution to detect the sequences
    # example:
    # x = [1, 0, 0, 1, 1, 1, 0]
    # x ⊙ [1, 1, 1] = [0, 0, 0, 3, 2]
    # note: mode='valid' only computes the convolution when there is an overlapping
    return sum(np.sum(np.convolve(kernel, seq, mode='valid') == length) for seq in sequences)


## Example

In [5]:
board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
play(board, 3, PLAYER_1)
play(board, 0, PLAYER_2)
play(board, 4, PLAYER_1)
play(board, 0, PLAYER_2)
play(board, 5, PLAYER_1)
play(board, 4, PLAYER_1)

pretty_print_board(board)
print(f"Number of sequences of length 3 (player 1): {count_by_length(board, 1, 3)}")


P1: X	P2: O
  ╔═════╦═════╦═════╦═════╦═════╦═════╦═════╗
5 ║     ║     ║     ║     ║     ║     ║     ║
  ╠═════╬═════╬═════╬═════╬═════╬═════╬═════╣
4 ║     ║     ║     ║     ║     ║     ║     ║
  ╠═════╬═════╬═════╬═════╬═════╬═════╬═════╣
3 ║     ║     ║     ║     ║     ║     ║     ║
  ╠═════╬═════╬═════╬═════╬═════╬═════╬═════╣
2 ║     ║     ║     ║     ║     ║     ║     ║
  ╠═════╬═════╬═════╬═════╬═════╬═════╬═════╣
1 ║  O  ║     ║     ║     ║  X  ║     ║     ║
  ╠═════╬═════╬═════╬═════╬═════╬═════╬═════╣
0 ║  O  ║     ║     ║  X  ║  X  ║  X  ║     ║
  ╚═════╩═════╩═════╩═════╩═════╩═════╩═════╝
     0     1     2     3     4     5     6    
Number of sequences of length 3 (player 1): 1


## Minimax

In [6]:
def vanilla_minimax(board: np.ndarray, player: int, minimize=True,
                    depth=3, *, score_func) -> Tuple[int, float]:
    """
    This function returns the next best move for the current player

    Parameters
    ----------
    board: array_like
        the connect 4 board
    player: int
        the next player to play
    minimize: boolean
        whether the player is a minimizer or a maximizer
    depth: int
        depth of the minimax search
    score_func: function
        an (heuristic) function that evaluates the board assigning it a score between -inf and +inf

    Returns
    -------
    int
        the suggested next move to play
    float
        the score resulting from playing the suggested move
    """

    moves = valid_moves(board)

    if count_by_length(board, player, 4):
        # penalize moves deep in the minimax recursion
        return None, WIN // (4 - depth)

    if count_by_length(board, -player, 4):
        # penalize moves deep in the minimax recursion
        return None, -WIN // (4 - depth)

    if depth == 0 or len(moves) == 0:
        # return None, score_func(board, player)
        return None, score_func(board, player)

    if depth and np.count_nonzero(board) < 2:
        return 3, 0

    # compute the scores for all the possibile moves
    scores = []

    for move in moves:
        # play in the column 'move'
        play(board, move, player)

        # update the score
        _, new_score = vanilla_minimax(
            board, -player, not minimize, depth - 1, score_func=score_func)

        # undo the move
        take_back(board, move)

        # append the score
        scores.append(new_score)

    best_score = min(scores) if minimize else max(scores)
    return moves[scores.index(best_score)], best_score


### Alpha-beta pruning

In [7]:
def alphabeta(board: np.ndarray, player: int, minimizer=True,
              depth: int = 3, alpha: float = -np.inf, beta:
              float = np.inf, *, score_func) -> Tuple[int, float]:
    """
    Compute the next best move for `player` using a minimax + alpha-beta pruning strategy.

    Parameters
    ----------
    board: array_like
        the connect 4 board
    player: int
        the next player to play
    minimize: boolean
        whether the player is a minimizer or a maximizer
    depth: int
        depth of the minmax search
    alpha: float
        the minimum score that the maximizing player is assured of
    beta: float
        the maximum score that the minimizing player is assured of
    score_func: function
        an (heuristic) function that evaluates the board assigning it a score between -inf and +inf

    Returns
    -------
    int
        the suggested next move to play
    float
        the score resulting from playing the suggested move
    """

    moves = valid_moves(board)

    if count_by_length(board, player, 4):
        # penalize moves deep in the minimax recursion
        return None, WIN // (4 - depth)

    if count_by_length(board, -player, 4):
        # penalize moves deep in the minimax recursion
        return None, -WIN // (4 - depth)

    if depth == 0 or len(moves) == 0:
        # return None, score_func(board, player)
        return None, score_func(board, player)

    if depth and np.count_nonzero(board) < 2:
        return 3, 0

    # The maximizer starts from best_score = -inf and it is assured
    # that the minimum value it can encounter is alpha.
    # On the other side, the minimizer starts from
    # best_score = inf and it is assured that the maximum value
    # it can encouter is beta
    best_move, best_score = None, np.inf if minimizer else -np.inf

    for move in moves:
        # play in the column 'move'
        play(board, move, player)

        # update the score
        _, new_score = alphabeta(board, -player, not minimizer,
                                 depth - 1, alpha, beta, score_func=score_func)

        take_back(board, move)

        if minimizer:
            if new_score < best_score:
                # better score found
                best_move, best_score = move, new_score

            beta = min(beta, best_score)
            if new_score < alpha:
                break  # no need to go further
        else:
            # maxmizer
            if new_score > best_score:
                # better score found
                best_move, best_score = move, new_score

            alpha = max(alpha, best_score)
            if new_score > beta:
                break  # no need to go further

    return best_move, best_score


## Static evaluation

A key for the success of a minimax strategy is the availability of a good static evaluation function that is able to make up for the limited depth of the search algorithm.

### Sequences extraction

A sequence of length n is a set of n discs arranged in an horizontal, vertical or diagonal line.

#### Fast convolution
Computing the sequences in the board is an expensive operation. The standard approach iterates over all the subsequences in the row|column|diagonal and checks whether at least one subsequence contains the same element.

A better approach convolves an array of length `n` and the subsequence and checks whether at least one value in the result is `n`. The convolution allows to obtain also the start position of each subsequence for free.
This approach is less expensive than the first one but convolution is still a lengthy operation. However, one could observe that all the rows, columns and diagonals of the board are simmetric with respect to the player. **Rows, columns and diagonals have at most length 7 and they admit only three values `[-1, 0, 1]`. Therefore, there are 2187 ($3^7$) different combinations. The proposed approach exploits this simmetry and precomputes the result of the convolution operation over all the combinations.**

`build_fast_convolution_cache` builds a tree where each node represents an entry of the row, column or diagonal and the leaves contain the result of the convolution of a length `n` ones array and the array given by all the nodes leading to that leaf. **Long story short, convolution now only requires the traversal of a tree of depth 7. This approach is $100$ times faster than the first approach.**

In [8]:
def build_fast_convolution_cache(depth: int, values: List[int] = []) -> dict:
    """
    Build the fast convolution cache

    Parameters
    ----------
    depth: int
        the depth in the fast convolution tree
    values: List[int]
        the values in the tree from the root down to this node
    """
    if depth == 0:
        array = np.array(values, dtype=np.byte)
        ones = np.ones(4, dtype=np.byte)

        return {
            player: {
                # cache the start position of each subsequence
                1: tuple(np.argwhere(np.convolve(player * ones[:1], array, mode='valid') == 1).flatten()),
                2: tuple(np.argwhere(np.convolve(player * ones[:2], array, mode='valid') == 2).flatten()),
                3: tuple(np.argwhere(np.convolve(player * ones[:3], array, mode='valid') == 3).flatten()),
                4: tuple(np.argwhere(np.convolve(player * ones[:4], array, mode='valid') == 4).flatten()),
            } for player in [-1, 1]
        }

    return {
        # we need to go deeper
        n: build_fast_convolution_cache(depth-1, values + [n]) for n in [-1, 0, 1]
    }


fast_convolution_dict = build_fast_convolution_cache(7)


def fast_convolve(row: np.ndarray):
    """
    Compute the convolution result by traversing the tree

    Parameters
    ----------
    row: np.ndarray
        the array to convolve with
    """
    _state = fast_convolution_dict
    # .tolist() is 10x faster than iterating directly on the array
    for digit in row.tolist():
        _state = _state[digit]

    for _ in range(7 - row.size):
        _state = _state[0]

    return _state


In [9]:
row = board[0]

# standar approach: iterate over all the subsequences of the row
%timeit any(all(row[r] == 1) for c in range(NUM_COLUMNS) for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1)))
# convolve [1, 1, 1, 1] and row
# then check whether at least one entry is equal to 4
%timeit np.any(np.convolve(np.ones(4), row) == 4)
# use fast convolution
%timeit len(fast_convolve(row)[1][4]) > 0

60.9 µs ± 417 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
8.52 µs ± 18.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
617 ns ± 0.272 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


#### Implementation

In [10]:
@dataclass
class Sequence:
    board: np.ndarray
    # start position of the sequence (example: [0, 0])
    start: np.ndarray
    # direction of the sequence (example: [0, 1] identifies a column growing bottom to top)
    step: np.ndarray
    # length of the sequence
    length: int

    def __len__(self):
        return self.length

    def __str__(self):
        return f"Sequence(start={self.start}, length={self.length}, step={self.step})"

    def __repr__(self) -> str:
        return self.__str__()


def compute_sequences(board: np.ndarray, player: int) -> List[Sequence]:
    """
    Compute all the sequences of length 1 (not really a sequence), 
    2, 3 and 4 in `board` played by `player`

    Parameters
    ----------
    board: np.ndarray
        the connect four board
    player: int
        the player whose sequences the function returns

    Returns
    -------
    List[Sequence]
        a list of sequences found in the board
    """
    diagspan = COLUMN_HEIGHT - 2

    lines = itertools.chain(
        # for each col take (start_position, direction, data)
        (((ncol, 0), (0, 1), col) for ncol, col in enumerate(board)),
        # for each row take (start_position, direction, data)
        (((0, nrow), (1, 0), row) for nrow, row in enumerate(board.T)),
        # for each diagonal take (start_position, direction, data)
        (((max(-ndiag, 0), max(ndiag, 0)), (1, 1), np.diag(board, ndiag))
         for ndiag in range(-diagspan, diagspan)),
        # for each antidiagonal take (start_position, direction, data)
        (((max(-ndiag, 0), min(COLUMN_HEIGHT - 1, COLUMN_HEIGHT - 1 - ndiag)), (1, -1),
         np.diag(np.fliplr(board), ndiag)) for ndiag in range(-diagspan, diagspan))
    )

    return [
        Sequence(board, 
            np.array(start) + index * np.array(step), # start point
            np.array(step), # direction 
            length # length of the sequence
        )
        for start, step, line in lines
        for length, indices in fast_convolve(line)[player].items()
        for index in indices # index is one of [1, 2, 3, 4]
    ]


In [11]:
compute_sequences(board, -1)

[Sequence(start=[0 0], length=1, step=[0 1]),
 Sequence(start=[0 1], length=1, step=[0 1]),
 Sequence(start=[0 0], length=2, step=[0 1]),
 Sequence(start=[0 0], length=1, step=[1 0]),
 Sequence(start=[0 1], length=1, step=[1 0]),
 Sequence(start=[0 0], length=1, step=[1 1]),
 Sequence(start=[0 1], length=1, step=[1 1])]

### An heuristic evaluation function

The proposed heuristic evaluation function computes all the sequences that can be generated by the next move of the player. The number of sequences for each different length are then weighted proportionally to their length and summed.

In [12]:
def heuristic_eval(board: np.ndarray, player: int) -> float:
    """
    Compute an heuristic score of the `board` assuming
    `player` is the next to play

    Parameters
    ----------
    board: np.ndarray
        the connect 4 board
    player: int
        the next player to play

    Returns
    -------
    float
        computed score
    """

    # check if the position is valid
    def is_valid(a, b): return 0 <= a < NUM_COLUMNS and 0 <= b < COLUMN_HEIGHT

    def is_free(a, b): return board[a, b] == 0
    def empty_below(a, b): return 0 if b == 0 else (board[a, :b] == 0).sum()

    # player
    player_score = 0
    for sequence in compute_sequences(board, player):
        # evaluate the sequence

        if len(sequence) == 4:
            # the player wins (return a big number)
            return WIN

        for nsteps in [-1, len(sequence)]:
            # take a step in one direction
            candidate = sequence.start + nsteps * sequence.step

            if is_valid(*candidate) and is_free(*candidate):
                # the position is valid and empty

                new_length = len(sequence) + 1

                # look a few steps ahead on the same direction
                # maybe filling this cell could join sequences
                for steps_ahead in range(2, 4 - new_length + 2):
                    a, b = candidate + steps_ahead * \
                        np.sign(nsteps) * sequence.step

                    if not is_valid(a, b) or not board[a, b] == player:
                        break
                    new_length += 1

                if new_length == 4 and empty_below(*candidate) == 0:
                    # current sequence has length 3 and can be extended
                    return WIN
                else:
                    player_score += new_length * \
                        (new_length - empty_below(*candidate))

    return player_score


## Monte Carlo Tree Search

Vanilla Monte Carlo Tree Search using UCT as a selection policy.

**Reference**: section 6.4 of *Stuart J. Russell, Peter Norvig. Artificial Intelligence: A Modern Approach (4th ed.)*

In [13]:
class MCTSNode:
    """
    A node in the Monte Carlo Tree Search algorithm.
    """

    def __init__(self, board: np.ndarray, player: int, move: int,
                 parent: Optional['MCTSNode'] = None) -> None:
        """
        Parameters
        ----------
        board: np.ndarray
            the connect 4 board
        player: int
            the player the played the last move
        move: int
            the last move
        parent: Optional['MCTSNode']
            the parent node in the tree
        """
        self.board = board
        self.player = player
        self.move = move
        self.utility = 0
        self.n_simulations = 0
        self.parent = parent
        self.children = []

    def propagate_utility_update(self, value: int):
        """
        Backpropagate a utility update up to the root node.
        Note that only the even nodes (from bottom to top) are updated.

        Parameters
        ----------
        value: int
            the update value
        """
        self.utility += value

        if self.parent:
            # update only the nodes of the player
            self.parent.propagate_utility_update(not value)

    def inc_num_simulations(self, value: int):
        """
        Increment the simulations counter up to the root node

        Parameters
        ----------
        value: int
            the update value
        """
        self.n_simulations += value

        if self.parent:
            self.parent.inc_num_simulations(value)

    def UCT(self, c: float = 2**0.5) -> float:
        """
        Compute the Upper Confidence Trees metric.
        
        Parameters
        ----------
        c: float
            the c parameter in the UCT formula

        Returns
        -------
        float
            the computed UCT metric
        """
        Ni = max(self.parent.n_simulations if self.parent else 0, 1)
        ni = max(self.n_simulations, 1)
        wi = max(self.utility, 1)

        return (wi / ni) + c * math.sqrt(max(math.log(Ni) / ni, 0))


def MCTS(board: np.ndarray, player: int,
         budget: int = 100, 
         playout_policy: Callable[[np.ndarray, int], int] = None) -> int:
    """
    Perform a Monte Carlo Tree Search to select the best next move
    
    Parameters
    ----------
    board: np.ndarray
        the connect 4 board
    player: int
        the next player to play
    budget: int
        the number of simulations to perform
    playout_policy: Callable[[np.ndarray, int], int]
        the playout policy (if none is provided, the function implements a naive player)
    
    Returns
    -------
    int
        the suggested next move

    """
    if not playout_policy:
        # no playout policy was provided => implement a naive player
        def playout_policy(board, _): 
            return np.random.choice(valid_moves(board))

    def select(root: MCTSNode) -> MCTSNode:
        """
        Select the node for the MCTS simulation.

        Parameters
        ----------
        root: MCTSNode
            the root node
        
        Returns
        -------
        MCTSNode
            next node to simulate
        """
        if len(root.children) == 0:
            return root

        return select(max(root.children, key=MCTSNode.UCT))

    def expand(board: np.ndarray, player: int, parent:MCTSNode=None):
        """
        Expand a MCTS node.

        Parameters
        ----------
        board: np.ndarray
            the connect 4 board
        player: int
            the next player to play
        parent: MCTS
            the parent node

        Returns
        -------
        List[MCTSNode]
            a list of children nodes
        """
        nodes = []
        # for each available move create a new node

        for move in valid_moves(board):
            leaf_board = board.copy()
            play(leaf_board, move, 1)
            nodes.append(MCTSNode(leaf_board, player, move, parent=parent))
        
        return nodes

    def simulate(board: np.ndarray, player: int) -> int:
        """
        Simulate a complete game starting from the current board.

        Parameters
        ----------
        board: np.ndarray
            the connect 4 board
        player: int
            the next player to play

        Returns
        -------
        int
            the winner of the game
        """
        p = -player

        # play until the game is over
        while valid_moves(board):
            p = -p

            # select a game using the playout policy
            c = playout_policy(board, p)

            if c is not None:
                play(board, c, p)

            # p wins?
            if count_by_length(board, p, 4):
                return p

        # draw
        return 0

    # nodes = expand(board, player)
    # print([node.UCT(c) for node in nodes])
    root = MCTSNode(board, -player, -1)

    while budget:
        # 1. selection
        candidate = select(root)

        # 2. expansion
        if candidate.n_simulations == 0:
            candidate.children = expand(
                candidate.board, -candidate.player, candidate)
            choice = np.random.randint(0, len(candidate.children))
            candidate = candidate.children[choice]

        # 3. simulations
        winner = simulate(candidate.board, candidate.player)

        # 4. backpropagation
        if winner == player:
            # the player won: increase the utility of all *its* nodes up to the root
            candidate.propagate_utility_update(candidate.player == player)
        candidate.inc_num_simulations(1)

        budget += -1

    # select the best node
    return max(root.children, key=lambda node: node.n_simulations).move


## Evaluation

In [14]:
def naive_player(board: np.ndarray):
    """
    Randomly select an column
    
    Parameters
    ----------
    board: np.ndarray
        the connect 4 board

    Returns
    -------
    int
        a valid move
    """
    return np.random.choice(valid_moves(board))


In [15]:
def play_game(initial_board: np.ndarray, initial_player: int,
              player_func: Callable[[np.ndarray], int],
              opponent_func: Callable[[np.ndarray], int],
              return_board=False) -> Tuple[int, Optional[np.ndarray]]:
    """
    Play a connect 4 game.

    Parameters
    ----------
    initial_board: np.ndarray
        the initial game board state
    initial_player: int
        the first player to play
    player_func: Callable[[np.ndarray], int]
        the function that implements the player's intelligence
    opponent_func: Callable[[np.ndarray], int]
        the function that implements the opponent's intelligence
    return_board: boolean
        whether the function should return the final board state

    Returns
    -------
    int
        the winner (one of -1, 0, 1)
    Optional[np.ndarray]
        the final board state
    """
    player = initial_player

    board = initial_board.copy()
    steps = 0

    while not(four_in_a_row(board, initial_player) or four_in_a_row(board, -initial_player)):
        # as long as there is not a winner keep playing

        if player == initial_player:
            # select a move for the player
            move = player_func(board)
        else:
            # select a move for the opponent
            move = opponent_func(board)

        if move is None:
            # no move available => exit
            break

        play(board, move, player)

        # switch to the other player
        player = player * -1
        steps = steps + 1

    # compute the game result (player wins, opponent wins, draw)
    winner = 0
    if count_by_length(board, -initial_player, 4):
        winner = -initial_player
    elif count_by_length(board, initial_player, 4):
        winner = initial_player

    if return_board:
        return winner, board
    return winner


In [16]:
def run_simulation(game_function: Callable[[], int], runs: int, player: int):
    """
    Perform repetead game simulations

    Parameters
    ----------
    game_function: Callable[[], int]
        a function that implements the game and returns the 
        result (one of -1, 0, 1)
    runs: int
        the number of simulations to be performed
    player:
        the player
    """
    print(f"Performing {runs} simulations...")

    simulation_results = {player: 0, -player: 0, 0: 0}

    start_time = perf_counter()

    for _ in tqdm(range(runs)):
        simulation_results[game_function()] += 1

    elapsed_time = perf_counter() - start_time

    wins_perc = 100 * (simulation_results[player] / runs)
    draw_perc = 100 * (simulation_results[0] / runs)

    print(f"Simulations completed in {elapsed_time:.6f} seconds!")
    print(f"Wins={wins_perc:.2f}, Draws={draw_perc:.2f}")


### Minimax

In [17]:
# wrap the heuristic evaluation function with a decorator that 
# counts the number of calls
board_evaluation_function = counted(heuristic_eval)

# reinitialize the random seed
np.random.seed(1234)

# initial board
board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
# initial player
player = PLAYER_1

# player function
player_func = lambda board: alphabeta(board, player, depth=3, score_func=board_evaluation_function)[0]
# opponent function
opponent_func = naive_player

# run a game
winner, final_board = play_game(board, player, player_func,
                                opponent_func, return_board=True)

print(f"{winner} wins! (n. of calls to the board evaluation function: {board_evaluation_function.calls})")
pretty_print_board(final_board)


1 wins! (n. of calls to the board evaluation function: 906)
P1: X	P2: O
  ╔═════╦═════╦═════╦═════╦═════╦═════╦═════╗
5 ║     ║     ║     ║     ║     ║     ║     ║
  ╠═════╬═════╬═════╬═════╬═════╬═════╬═════╣
4 ║     ║     ║     ║     ║     ║     ║     ║
  ╠═════╬═════╬═════╬═════╬═════╬═════╬═════╣
3 ║     ║     ║     ║     ║  O  ║     ║     ║
  ╠═════╬═════╬═════╬═════╬═════╬═════╬═════╣
2 ║     ║     ║     ║  X  ║  X  ║     ║     ║
  ╠═════╬═════╬═════╬═════╬═════╬═════╬═════╣
1 ║     ║     ║     ║  O  ║  O  ║     ║  X  ║
  ╠═════╬═════╬═════╬═════╬═════╬═════╬═════╣
0 ║  O  ║  X  ║  X  ║  X  ║  X  ║  O  ║  O  ║
  ╚═════╩═════╩═════╩═════╩═════╩═════╩═════╝
     0     1     2     3     4     5     6    


In [18]:
# run a batch of simulations
run_simulation(
    functools.partial(play_game, board, player, player_func, opponent_func),
    500, # number of simulations to perform
    player # initial player
)

Performing 500 simulations...


100%|██████████| 500/500 [09:56<00:00,  1.19s/it]

Simulations completed in 596.026520 seconds!
Wins=99.40, Draws=0.00


### Monte Carlo Tree Search

Monte Carlo Tree Search is definitely slow. In order to reasonably limit the execution time, the MCTS algorithm is limited to a depth of 50. The playout policy implements a simple alpha-beta minimax with depth set to 1 (or 2 if you have enough time).

In [19]:
def playout_policy(board: np.ndarray, player: int):
    """
    Playout policy for the MCTS algorithm that
    implements a simple alpha-beta minimax approach
    """

    def score_function(board: np.ndarray, player: int):
        """
        Simple score function that counts the 
        number of sequences of length 3
        """
        # same as:
        return count_by_length(board, player, 3) - count_by_length(board, -player, 3)

    # alphabeta returns (suggested_move, new_score) => return only the move
    return alphabeta(board, player, depth=2, score_func=score_function)[0]


In [20]:
# reinitialize the random seed
np.random.seed(1234)

# initial board
board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
# initial player
player = PLAYER_1

# player function
player_func = functools.partial(MCTS, player=player,
                                budget=50, playout_policy=playout_policy)
# opponent function
opponent_func = naive_player

# run a game
winner, final_board = play_game(board, player, player_func,
                                opponent_func, return_board=True)
pretty_print_board(final_board)


P1: X	P2: O
  ╔═════╦═════╦═════╦═════╦═════╦═════╦═════╗
5 ║  O  ║  O  ║     ║     ║     ║     ║     ║
  ╠═════╬═════╬═════╬═════╬═════╬═════╬═════╣
4 ║  X  ║  X  ║     ║     ║     ║     ║     ║
  ╠═════╬═════╬═════╬═════╬═════╬═════╬═════╣
3 ║  O  ║  O  ║     ║  O  ║     ║     ║     ║
  ╠═════╬═════╬═════╬═════╬═════╬═════╬═════╣
2 ║  X  ║  X  ║  X  ║  X  ║     ║     ║     ║
  ╠═════╬═════╬═════╬═════╬═════╬═════╬═════╣
1 ║  X  ║  O  ║  X  ║  X  ║     ║     ║  O  ║
  ╠═════╬═════╬═════╬═════╬═════╬═════╬═════╣
0 ║  O  ║  X  ║  X  ║  O  ║  X  ║  O  ║  O  ║
  ╚═════╩═════╩═════╩═════╩═════╩═════╩═════╝
     0     1     2     3     4     5     6    


In [21]:
# run a batch of simulations
run_simulation(
    lambda: play_game(board, player, player_func, opponent_func),
    10, # number of simulations to perform
    player # initial player
)

Performing 10 simulations...


100%|██████████| 10/10 [10:59<00:00, 65.96s/it]

Simulations completed in 659.632352 seconds!
Wins=100.00, Draws=0.00


## Conclusions
The **minimax** and **minimax + alpha-beta pruning** are very fast but they may rarely perform sub-optimally (on the order of 1-2% losses versus a naive player). Further fine tuning of the heuristic evaluation function could probably bring this value (close) to zero.

The **MCTS** approach appears to be way stronger despite being terribly slow (even with a small simulations budget).